In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gseapy as gp

In [2]:
mgl_genes = pd.read_csv("/mnt/c/Users/ouyangkang/Downloads/microglia_markers.csv")
mac_genes = pd.read_csv("/mnt/c/Users/ouyangkang/Downloads/macrophage_markers.csv")

In [3]:
print(mgl_genes.shape)
print(mac_genes.shape)
mac_genes.head()

(29, 10)
(158, 10)


,Unnamed: 0,seqnames,start,end,strand,name,idx,Log2FC,FDR,MeanDiff
0,18092,chr6,108489743,108489803,1,Mir7661,805,1.790966,9.062958e-46,1.202686
1,18093,chr6,108660934,108577036,2,0610040F04Rik,806,1.312830,1.023321e-39,0.710494
2,18240,chr6,122626424,122630271,1,Dppa3,953,1.660713,3.140691e-39,0.790734
3,18241,chr6,122707489,122714633,1,Nanog,954,1.576998,4.150824e-38,0.455416
4,743,chr1,128592299,128588199,2,Cxcr4,743,1.639783,2.213639e-27,1.046892


In [4]:
gp.get_library_name(organism='Human')

['ARCHS4_Cell-lines',
 'ARCHS4_IDG_Coexp',
 'ARCHS4_Kinases_Coexp',
 'ARCHS4_TFs_Coexp',
 'ARCHS4_Tissues',
 'Achilles_fitness_decrease',
 'Achilles_fitness_increase',
 'Aging_Perturbations_from_GEO_down',
 'Aging_Perturbations_from_GEO_up',
 'Allen_Brain_Atlas_10x_scRNA_2021',
 'Allen_Brain_Atlas_down',
 'Allen_Brain_Atlas_up',
 'Azimuth_2023',
 'Azimuth_Cell_Types_2021',
 'BioCarta_2013',
 'BioCarta_2015',
 'BioCarta_2016',
 'BioPlanet_2019',
 'BioPlex_2017',
 'CCLE_Proteomics_2020',
 'CORUM',
 'COVID-19_Related_Gene_Sets',
 'COVID-19_Related_Gene_Sets_2021',
 'Cancer_Cell_Line_Encyclopedia',
 'CellMarker_2024',
 'CellMarker_Augmented_2021',
 'ChEA_2013',
 'ChEA_2015',
 'ChEA_2016',
 'ChEA_2022',
 'Chromosome_Location',
 'Chromosome_Location_hg19',
 'ClinVar_2019',
 'DGIdb_Drug_Targets_2024',
 'DSigDB',
 'Data_Acquisition_Method_Most_Popular_Genes',
 'DepMap_CRISPR_GeneDependency_CellLines_2023',
 'DepMap_WG_CRISPR_Screens_Broad_CellLines_2019',
 'DepMap_WG_CRISPR_Screens_Sanger_Cell

In [5]:
def enrichment(gene_list, file_name_prefix):
    # enrichment analysis
    enr = gp.enrich(gene_list,
                    gene_sets="GO_Biological_Process_2023",
                    outdir=None)
    # filtering p value is more than 0.05
    result = enr.res2d[enr.res2d['P-value'] < 0.05]
    # save result
    result.to_csv(f"/mnt/c/Users/ouyangkang/Desktop/{file_name_prefix}_result.csv", index=None)

    return result

In [6]:
mgl_result = enrichment(mgl_genes.name.tolist(), 'mgl_diff')
mac_result = enrichment(mac_genes.name.tolist(), 'mac_diff')

In [7]:
mgl_result.head()

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,GO_Biological_Process_2023,Amylin Receptor Signaling Pathway (GO:0097647),1/5,0.007230,0.12275,0,0,178.276786,878.827148,RAMP2
1,GO_Biological_Process_2023,Vascular Associated Smooth Muscle Cell Develop...,1/6,0.008670,0.12275,0,0,142.614286,677.124522,RAMP2
2,GO_Biological_Process_2023,Calcitonin Family Receptor Signaling Pathway (...,1/6,0.008670,0.12275,0,0,142.614286,677.124522,RAMP2
3,GO_Biological_Process_2023,Astral Microtubule Organization (GO:0030953),1/8,0.011543,0.12275,0,0,101.857143,454.451827,TPPP
4,GO_Biological_Process_2023,Vascular Associated Smooth Muscle Cell Differe...,1/8,0.011543,0.12275,0,0,101.857143,454.451827,RAMP2


In [8]:
select_mgl = mgl_result.iloc[[0, 1, 2 ,5, 8, 14, 15, 18, 19, 22]]
select_mgl["Lable"] = "MGL"
select_mac = mac_result.iloc[[0, 1, 2, 3, 5, 6, 8, 16, 24, 39]]
select_mac["Lable"] = "MAC"

/tmp/ipykernel_657/870613516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_mgl["Lable"] = "MGL"
/tmp/ipykernel_657/870613516.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_mac["Lable"] = "MAC"


In [9]:
select_all = pd.concat([select_mac, select_mgl], ignore_index=True)
select_all.Term = select_all.Term.str.split("(").str[0]
select_all.head()

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Lable
0,GO_Biological_Process_2023,Inflammatory Response,9/236,0.000111,0.075060,0,0,5.219377,47.511134,IL22;HP;FPR1;CYBB;CXCR4;CXCL1;ITGAL;CXCL2;S100A8,MAC
1,GO_Biological_Process_2023,Neutrophil Chemotaxis,5/70,0.000231,0.075060,0,0,9.943188,83.252182,LGALS3;SAA1;CXCL1;CXCL2;S100A8,MAC
2,GO_Biological_Process_2023,Granulocyte Chemotaxis,5/73,0.000281,0.075060,0,0,9.503076,77.695580,LGALS3;SAA1;CXCL1;CXCL2;S100A8,MAC
3,GO_Biological_Process_2023,Neutrophil Migration,5/77,0.000361,0.075060,0,0,8.973312,71.131523,LGALS3;SAA1;CXCL1;CXCL2;S100A8,MAC
4,GO_Biological_Process_2023,Histidine Metabolic Process,2/6,0.000911,0.126335,0,0,63.583333,445.140237,HAL;HDC,MAC


In [10]:
gp.barplot(
    select_all,
    figsize=(6, 8),
    title="Different function of DEGs between MGL and MAC",
    group="Lable",
    color=["b", "r"],
    column="P-value",
    ofname = "MGL-MAC_diff_function.pdf"
)

/home/ouyangkang/software/miniconda3/envs/sc/lib/python3.11/site-packages/gseapy/plot.py:694: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df[self.colname].replace(
/home/ouyangkang/software/miniconda3/envs/sc/lib/python3.11/site-packages/gseapy/plot.py:694: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[self.colname].replace(
